In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from bs4 import BeautifulSoup
import re
import joblib
import os
import glob
import pandas as pd

In [2]:
# documento = '477918-calentamiento-global-la-comunidad-cientifica-internacional-l.html'
ruta_documentos = r'C:\Users\nacho\My Drive\3.- MCD\3.- Materias\18.- Web Mining\Práctica\TP1\Articulos_test_ultima_semana'

## Funciones

In [3]:
def leer_html(documento):
    """
    Lee documentos html.
    """
    DOCUMENTO = open(documento,"rt", encoding="utf-8").read() # se agrega el encoding.
    return DOCUMENTO

In [4]:
def extractor(delimitador, doc_html):
    """
    Estrae texto del html a partir del delimitador que se use.
    delimitador: expresión usada para busqueda del fragmento de texto a extraer.
    doc_html: documento sobre el cual se extraerá el texto.
    Nota: beautifulsoup hace todo el trabajo de ignorar los tags de html y convertir las entidades (p.ej. &ntilde;) a letras (p.ej. ñ)
    """
    try:
        match = delimitador.search(doc_html.replace("\n",""))
        extractor_html = BeautifulSoup(match.group(1), 'html.parser')
        texto = extractor_html.get_text(separator=" ", strip=True)
    except:
        texto = None
    
    return texto

In [5]:
def lista_archivos(ruta_documentos):
    archivos = []
    # search all files inside a specific folder
    # *.* means file name with any extension
    dir_path = ruta_documentos + '\**\*.html'
    for file in glob.glob(dir_path, recursive=True):
        archivos.append(file)
    return archivos

In [6]:
def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

## Delimitadores para búsqueda en html

In [7]:
# Delimitadores de la fecha que se quiere extraer:
MARCADOR_COMIENZO_FECHA = '<time datetime=\"'
MARCADOR_FIN_FECHA = r'T\d{2}:\d{2}:\d{2}-03:00'

In [8]:
# Delimitadores del texto que se quiere extraer:
MARCADOR_COMIENZO_INTERESANTE="<p>"
MARCADOR_FIN_INTERESANTE="</main>"

In [9]:
fecha_interes = re.compile(re.escape(MARCADOR_COMIENZO_FECHA) + '(.+)' + MARCADOR_FIN_FECHA)
texto_interes = re.compile(re.escape(MARCADOR_COMIENZO_INTERESANTE) +'(.+)'+ re.escape(MARCADOR_FIN_INTERESANTE))

## Cálculo 

In [10]:
fechas = []
textos = []
targets = []
notas = []
temas= get_immediate_subdirectories(ruta_documentos)
nota_problema = []
fecha_problema = []
tema_problema = []

for documento in lista_archivos(ruta_documentos):

    for tema in temas:
        if f"\\{tema}\\" in documento:
            target = tema
    # print(documento)
    # print(target)        
    fecha = extractor(fecha_interes,leer_html(documento))
    texto = extractor(texto_interes,leer_html(documento))
            
    if texto is None:
        fecha_problema.append(fecha)
        tema_problema.append(target)
        nota_problema.append(os.path.basename(documento))
    else:
        # print(fecha)
        # print(target)
        # print(documento)
        fechas.append(fecha)
        textos.append(texto)
        targets.append(target)
        notas.append(os.path.basename(documento))

In [11]:
# ruta_prueba = r'C:\Users\nacho\My Drive\3.- MCD\3.- Materias\18.- Web Mining\Práctica\TP1\Articulos\sociedad\437675-hay-casi-tres-mil-camiones-varados-en-mendoza.html'
# texto = extractor(texto_interes,leer_html(ruta_prueba))

In [12]:
datos = {'FECHA':fechas, 'TARGET':targets ,'NOTA':notas,'TEXTO':textos}
data = pd.DataFrame(datos)

In [13]:
problemas = {'FECHA':fecha_problema, 'TARGET':tema_problema, 'NOTA':nota_problema}
problema = pd.DataFrame(problemas)

In [14]:
data.shape

(263, 4)

In [16]:
data.head()

,FECHA,TARGET,NOTA,TEXTO
0,2022-09-05,economia,478671-cuenta-dni-que-es-y-como-funciona-la-pl...,La aplicación Cuenta DNI del Banco Provincia d...
1,2022-09-04,economia,479043-massa-a-ee-uu-viaje-geopolitico-dolares...,Casi al lado del despacho del ministro de Econ...
2,2022-09-04,economia,479068-dolares-por-que-paso-donde-estan-y-que-...,La balanza comercial de los últimos dos meses ...
3,2022-09-04,economia,479133-un-desierto-de-dolares-3000-metros-bajo...,Desde Neuquén La llegada a Vaca Muerta se intu...
4,2022-09-04,economia,479163-desde-este-lunes-los-cigarrillos-aument...,"Desde este lunes 5 de septiembre, todos los pr..."


In [17]:
data[['TARGET','NOTA']].groupby(by='TARGET').count()

,NOTA
TARGET,
economia,66
el-mundo,65
el-pais,66
sociedad,66


In [19]:
data.NOTA.unique().shape

(263,)

In [35]:
data[data.TEXTO.str.contains('ImageObject') == True]

,FECHA,TARGET,NOTA,TEXTO
87,2022-09-04,el-mundo,479203-chile-vota-el-destino-de-una-nueva-cons...,"El presidente de Colombia, Gustavo Petro , rea..."


In [34]:
data.TEXTO[87]

'El presidente de Colombia, Gustavo Petro , reaccionó este domingo al resultado del plebiscito en Chile . \\" Revivió Pinochet \\", escribió en Twitter. \\"Sólo si las fuerzas democráticas y sociales se unen, será posible dejar atrás un pasado que mancha a toda América Latina y abrir las alamedas democráticas\\", agregó el mandatario colombiano. En Twitter también se expresó el antecesor de Petro, Iván Duque . El expresidente de derecha, escribió que \\"habló el pueblo chileno en una bella jornada democrática\\". \\"Triunfó la sensatez sobre las imposiciones ideologizas que fracturarían para siempre una sociedad que ha alcanzado importantes logros sociales y económicos\\". ","mainEntityOfPage":{"@type":"WebPage"},"publisher":{"@type":"Organization","@id":"#PublisherOrg","name":"Página12","logo":{"@type":"ImageObject","url":"https://www.pagina12.com.ar/assets/media/logo_default_p12.png","width":"600","height":"60"}},"image":{"@type":"ImageObject","url":"https://www.pagina12.com.ar/asset

In [15]:
data.to_csv('Articulos_test_ultima_semana.csv',index=False)